In [23]:
import tensorflow as tf
import numpy as np
from c3.utils.qt_utils import basis, single_length_RB

In [6]:
from single_qubit_blackbox_exp import create_experiment

blackbox = create_experiment()

In [10]:
print(blackbox.gateset.print_parameters())

Id-d1-no_drive-t_final                : 7.000 ns 
Id-d1-no_drive-amp                    : 0.000 V 
Id-d1-no_drive-delta                  : 0.000 V 
Id-d1-no_drive-freq_offset            : 0.000 Hz 2pi 
Id-d1-no_drive-xy_angle               : 0.000 rad 
Id-d1-carrier-freq                    : 5.050 GHz 2pi 
Id-d1-carrier-framechange             : 4.084 rad 
X90p-d1-gauss-amp                     : 450.000 mV 
X90p-d1-gauss-t_final                 : 7.000 ns 
X90p-d1-gauss-sigma                   : 1.750 ns 
X90p-d1-gauss-xy_angle                : -444.089 arad 
X90p-d1-gauss-freq_offset             : -50.500 MHz 2pi 
X90p-d1-gauss-delta                   : -1.000  
X90p-d1-carrier-freq                  : 5.050 GHz 2pi 
X90p-d1-carrier-framechange           : 0.000 rad 
Y90p-d1-gauss-amp                     : 450.000 mV 
Y90p-d1-gauss-t_final                 : 7.000 ns 
Y90p-d1-gauss-sigma                   : 1.750 ns 
Y90p-d1-gauss-xy_angle                : 1.571 rad 
Y90p-d1-gauss-freq_

In [11]:
blackbox.get_gates()

{'Id': <tf.Tensor: shape=(3, 3), dtype=complex128, numpy=
 array([[ 1.        +0.00000000e+00j,  0.        +0.00000000e+00j,
          0.        +0.00000000e+00j],
        [ 0.        +0.00000000e+00j,  1.        -3.95017352e-13j,
          0.        +0.00000000e+00j],
        [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         -0.98228725+1.87381316e-01j]])>,
 'X90p': <tf.Tensor: shape=(3, 3), dtype=complex128, numpy=
 array([[ 0.7113148 -0.08098006j, -0.03515163-0.69637989j,
          0.03284002+0.01464368j],
        [-0.09987753-0.68998106j,  0.71416569-0.0586841j ,
         -0.00925939-0.01970837j],
        [ 0.03614727-0.01083843j, -0.01725489-0.00672552j,
         -0.99733698+0.05959769j]])>,
 'Y90p': <tf.Tensor: shape=(3, 3), dtype=complex128, numpy=
 array([[ 0.71182021-0.08071269j, -0.69587623+0.03552774j,
         -0.03282141-0.01463745j],
        [ 0.6895124 -0.0994323j ,  0.71462931-0.05905769j,
         -0.02076395+0.008477j  ],
        [-0.03675419+0.01150

In [15]:
blackbox.evaluate([["X90p", "X90p"]*2])

[<tf.Tensor: shape=(3, 1), dtype=float64, numpy=
 array([[0.9953031 ],
        [0.00292125],
        [0.00177564]])>]

In [34]:
min_length = 5
max_length = 500
num_lengths = 50
num_seqs = 30

psi_init = tf.constant(basis(3, 0), dtype=tf.complex128)
lengths = np.rint(np.linspace(min_length, max_length, num=num_lengths)).astype(int)
surv_prob = []
for L in lengths:
    seqs = single_length_RB(num_seqs, L)
    pops = blackbox.evaluate(seqs)
    surv_prob.append(pops[0][0])

In [28]:
surv_prob

[<tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.9913892])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.83948493])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.93912032])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.64996066])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.83778445])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.64454308])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.3965748])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.75048128])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.35819127])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.76398172])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.77433846])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.13564257])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.47700743])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.67748326])>,
 <tf.Tensor: shape=(1,), dtype=float64, numpy=arra

In [35]:
from scipy.optimize import curve_fit
def RB_fit(len, r, A, B):
    return A * r**(len) + B
bounds = (0, 1)
init_guess = [0.9, 0.5, 0.5]
means = np.mean(surv_prob, axis=1)
stds = np.std(surv_prob, axis=1) / np.sqrt(len(surv_prob[0]))
solution, cov = curve_fit(RB_fit,
                          lengths,
                          means,
                          sigma=stds,
                          bounds=bounds,
                          p0=init_guess)
r, A, B = solution

epc = 0.5 * (1 - r)
epg = 1 - ((1-epc)**(1/4))

ValueError: Residuals are not finite in the initial point.

In [ ]:
fig, ax = plt.subplots()
ax.plot(lengths,
        surv_prob,
        marker='o',
        color='red',
        linestyle='None')
ax.errorbar(lengths,
            means,
            yerr=stds,
            color='blue',
            marker='x',
            linestyle='None')
plt.title('RB results')
plt.ylabel('Population in 0')
plt.xlabel('\# Cliffords')
plt.ylim(0, 1)
plt.xlim(0, lengths[-1])
fitted = RB_fit(lengths, r, A, B)
ax.plot(lengths, fitted)
plt.text(0.1, 0.1,
         'r={:.4f}, A={:.3f}, B={:.3f}'.format(r, A, B),
         size=16,
         transform=ax.transAxes)
plt.show()